Import Pre-trained Local Llama3.1:8B Model

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

#Load Pre trained Local LLM. Varient of Llama3.1 named 
MODEL = "User_Manual_Expert_llama3.1"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()


**Now we load the pdf and load it in memory**


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('<Bike/Car_User_Manual>.pdf')
pages = loader.load_and_split()
#check pages
pages

**Make a chain to pass multiple parameters to invoke the model and parser the statement to only print relevant information**

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser

**Make a prompt template**

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
#test prompt with context and question
print(prompt.format(context="Here is the context", question="Here is the question"))


Answer the question based on the context below. If you can't answer the question reply "I don't know".

Context: Here is the context

Question: Here is the question



**We can check the chain schema to see hat variables are required**

In [9]:
chain = prompt | model
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

**Now to search for questions in the PDF we will first want to store the relevant information as per the context in a vector storage. To find the relevant storage we will make a embedding using Ollama and make a vector storage**

In [14]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding = embeddings
)

**To provide a better context for the LLM Model to work on we can retrive the relavant data related to the topic we are searching for and then use it to find any questions/topics related to it**

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("<..Specific Topic..>")

**Now Lets send the context and question to the LLM to check the output**

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question": "..Ask You Question Here.."})